In [1]:
import os
import wx
import time

#still needs to add three buttons
#for stride size, patch size, color --> bar (0~1)
class LoadImage(wx.App):
    def __init__(self, redirect=False, filename=None):
        wx.App.__init__(self, redirect, filename)
        self.frame = wx.Frame(None, title='Krypton')
        self.panel = wx.Panel(self.frame)
        self.PhotoMaxSize = 224
        self.createWidgets()
        self.frame.Show()
 
    def createWidgets(self):
        instructions = 'Select an image'
        img = wx.EmptyImage(224,224)
        self.imageCtrl = wx.StaticBitmap(self.panel, wx.ID_ANY, 
                                         wx.BitmapFromImage(img))
 
        instructLbl = wx.StaticText(self.panel, label=instructions)
        self.photoTxt = wx.TextCtrl(self.panel, size=(200,-1))
        browseBtn = wx.Button(self.panel, label='Browse')
        browseBtn.Bind(wx.EVT_BUTTON, self.onBrowse)
         
        run_button = wx.Button(self.panel, label='run',pos=(225, 50), size=(60, 25))    
        run_button.Bind(wx.EVT_PAINT, self.runModel)    
            
        self.patch_list = ['1', '2', '4', '8']
        self.stride_list = ['1', '2', '3', '4']
        self.model_list = ['VGG16','InceptionV3','ResNet']
        self.patch_size = wx.ListBox(self.panel, wx.ID_ANY, (300, 30), (50,50), self.patch_list, wx.LB_SINGLE, name = 'patch')
        self.stride_size = wx.ListBox(self.panel, wx.ID_ANY, (300, 90), (50,50), self.stride_list, wx.LB_SINGLE, name = 'stride')
        self.models = wx.ListBox(self.panel, wx.ID_ANY, (300, 150), (50,80), self.model_list, wx.LB_SINGLE, name = 'models')

        self.slider = wx.Slider(self.panel, wx.ID_ANY, value = 0.5, minValue = 0, maxValue = 1, name = 'color')

        self.mainSizer = wx.BoxSizer(wx.VERTICAL)
        self.sizer = wx.BoxSizer(wx.HORIZONTAL)
 
        self.mainSizer.Add(wx.StaticLine(self.panel, wx.ID_ANY),
                           0, wx.ALL|wx.EXPAND, 5)
        self.mainSizer.Add(instructLbl, 0, wx.ALL, 5)
        self.sizer.Add(self.slider, 0, wx.ALL, 5)
#         self.sizer.Add(patch_size, 0, wx.ALL, 5)
        self.mainSizer.Add(self.imageCtrl, 0, wx.ALL, 5)
        self.sizer.Add(self.photoTxt, 0, wx.ALL, 5)
        self.sizer.Add(browseBtn, 0, wx.ALL, 5)        
        self.mainSizer.Add(self.sizer, 0, wx.ALL, 5)
 
        self.panel.SetSizer(self.mainSizer)
        self.mainSizer.Fit(self.frame)
 
        self.panel.Layout()
 
    def onBrowse(self, event):
        """ 
        Browse for file
        """
        wildcard = "JPEG files (*.jpg)|*.jpg"
        dialog = wx.FileDialog(None, "Choose a file",
                               wildcard=wildcard,
                               style=wx.FD_OPEN)
        if dialog.ShowModal() == wx.ID_OK:
            self.photoTxt.SetValue(dialog.GetPath())
        dialog.Destroy() 
        self.onView()
 
    def onView(self):
        filepath = self.photoTxt.GetValue()
        img = wx.Image(filepath, wx.BITMAP_TYPE_ANY)
        # scale the image, preserving the aspect ratio
        W = img.GetWidth()
        H = img.GetHeight()
        if W > H:
            NewW = self.PhotoMaxSize
            NewH = self.PhotoMaxSize * H / W
        else:
            NewH = self.PhotoMaxSize
            NewW = self.PhotoMaxSize * W / H
        img = img.Scale(NewW,NewH)
 
        self.imageCtrl.SetBitmap(wx.BitmapFromImage(img))
        self.panel.Refresh()
        
    #i dont know what is this event for
    def runModel(self, event):
        patch_val = self.patch_list[self.patch_size.GetSelection()]
        stride_val = self.stride_list[self.stride_size.GetSelection()]
        model = self.model_list[self.models.GetSelection()]
        color = self.slider.GetValue()
        #run the Incremental Inference Krypton Model
        begin_time = time.time()
        heatmap, prob, label = inc_inference(model, self.photoTxt, patch_size= patch_val, stride=stride_val, beta=1.0, gpu=True, c=color)
        end_time = time.time()
        
        #then display the image
#         print('Predicted Label: %s, Probability: %f, Time: %f s'%(class_names[label], prob, end_time-begin_time))
#         show_heatmap(image_file_path, heatmap, label=class_names[label], prob=prob, width=224)

if __name__ == '__main__':
    app = LoadImage()
    app.MainLoop()

c:\users\cywkh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:18: wxPyDeprecationWarning: Call to deprecated item EmptyImage. Use :class:`Image` instead.
c:\users\cywkh\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:20: wxPyDeprecationWarning: Call to deprecated item BitmapFromImage. Use :class:`wx.Bitmap` instead


NameError: name 'inc_inference' is not defined